This part is reproduction of a previous work of past year student, Timothy Yeo

With 2 Data Center, each with 5 machines and 6 queue length.
total feature nubmer is 2 * (5 * 2+6 * 3+2) = 60

Single-Agent Baseline

In [351]:
import torch

# state: array of length 60, 30 for each data center
# action: 1 of the 4 combinations possible for 2 data centers

data_center_num = 2
machine_num = 5
queue_num = 6

def machine_index(data_center_id, machine_id):
    return data_center_id * (machine_num * 2 + queue_num * 3 + 2) + machine_id * 2

def queue_index(data_center_id, queue_id):
    return data_center_id * (machine_num * 2 + queue_num * 3 + 2) + machine_num * 2 + queue_id * 3

def incoming_index(data_center_id):
    return data_center_id * (machine_num * 2 + queue_num * 3 + 2) + machine_num * 2 + queue_num * 3

def step(state, action):
    state = state.detach().cpu().numpy()
    global record
    reward = 0
    index = 0

    for centerId in range(data_center_num):
        # update reward by queueing delay
        for queue in range(queue_num):
            index = queue_index(centerId, queue)
            if state[index] > 0:
                state[index + 2] = max(0, state[index + 2] - 0.1)

    

    
    # STEP 1: update machine states
    for centerId in range(data_center_num):
        # update machine states (timestep--)
        for machine in range(machine_num):
            index = machine_index(centerId, machine)
            if state[index] == 1:
                # decrease by 1 timestep
                state[index + 1] -= 1
                if state[index + 1] == 0:
                    state[index] = 0
                    
    # STEP 2: update queue states    
    for centerId in range(data_center_num):
        # update queue states (put into machines)
        for queue in range(queue_num):
            index = queue_index(centerId, queue)
            # if job exist
            if state[index] > 0:
                for machine in range(machine_num):
                    m_index = machine_index(centerId, machine)
                    if state[m_index] == 0:
                        state[m_index] = 1
                        state[m_index + 1] = state[index + 1]
                        state[index] = 0
                        # update reward
                        reward += state[index + 2]
                        record[state[index+2].item()] += 1
                        break

    action = torch.argmax(action)
    # update incoming jobs (locally)
    for job in range(data_center_num):
        if action // pow(2, job) % 2 == 0:
            for queue in range(queue_num):
                index = queue_index(job, queue)
                if state[index] == 0:
                    job_index = incoming_index(job)
                    state[index] = 1
                    state[index + 1] = state[job_index]
                    state[index + 2] = state[job_index + 1]
                    break
            else:
                record["abandoned"] += 1

    # update incoming jobs (remotely)
    for job in range(data_center_num):
        if action // pow(2, job) % 2 == 1:
            for queue in range(queue_num):
                index = queue_index(1- job, queue) # todo: fix logic for N > 2 ...
                if state[index] == 0:
                    job_index = incoming_index(job)
                    state[index] = 1
                    state[index + 1] = state[job_index]
                    state[index + 2] = state[job_index + 1] - 0.2
                    record["remote"] += 1
                    break
            else:
                record["abandoned"] += 1
    

    # STEP 2: update queue states    
    for centerId in range(data_center_num):
        # update queue states (put into machines)
        for queue in range(queue_num):
            index = queue_index(centerId, queue)
            # if job exist
            if state[index] > 0:
                for machine in range(machine_num):
                    m_index = machine_index(centerId, machine)
                    if state[m_index] == 0:
                        state[m_index] = 1
                        state[m_index + 1] = state[index + 1]
                        state[index] = 0
                        # update reward
                        reward += state[index + 2]
                        record[state[index+2].item()] += 1
                        break


    # update new incoming job
    for job in range(data_center_num):
        job_index = incoming_index(job)
        seed = torch.rand(1).item()
        if seed < 0.4:
            state[job_index] = 8
            state[job_index + 1] = 0.8
        elif seed < 0.7:
            state[job_index] = 4
            state[job_index + 1] = 0.4
        else:
            state[job_index] = 2
            state[job_index + 1] = 0.2

    return state, reward

        

In [352]:
from collections import Counter
record = Counter()


start = torch.zeros(60)
start[0] = 1
start[1] = 4
start[2] = 1
start[3] = 1

start[queue_index(0,0)] = 1
start[queue_index(0,0) + 1] = 5
start[queue_index(0,0) + 2] = 12

action = torch.tensor([0, 1, 0, 0])

print(step(start, action))
print(record)

(array([ 1. ,  3. ,  1. ,  5. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
        5. , 11.9,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
        0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  8. ,  0.8,  1. ,  0. ,  1. ,
        0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
        0. , -0.2,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
        0. ,  0. ,  0. ,  2. ,  0.2], dtype=float32), 11.699999615550041)
Counter({11.899999618530273: 1, 'remote': 1, 0.0: 1, -0.20000000298023224: 1})


In [353]:
# create a NN network with following architecture
# Input Layer (60 features)
# Leaky ReLU activation
# Hidden Layer 1 (512 features)
# Leaky ReLU activation
# Hidden Layer 2 (256 features)
# Leaky ReLU activation
# Hidden Layer 3 (256 features)
# Leaky ReLU activation
# Hidden Layer 3 (256 features)
# Leaky ReLU activation
# Output Layer (4 actions)

import numpy as np
import torch
import torch.nn as nn

class SingleAgentNN(nn.Module):
    def __init__(self):
        super(SingleAgentNN, self).__init__()
        self.layer1 = nn.Linear(60, 512)
        self.layer2 = nn.Linear(512, 256)
        self.layer3 = nn.Linear(256, 256)
        self.layer4 = nn.Linear(256, 256)
        self.layer5 = nn.Linear(256, 4)

    def __str__(self):
        return f'Neural Network with input layer {self.input_layer}, hidden layer 1 {self.hidden_layer_1}, hidden layer 2 {self.hidden_layer_2}, hidden layer 3 {self.hidden_layer_3}, hidden layer 4 {self.hidden_layer_4}, and output layer {self.output_layer}'

    def __repr__(self):
        return self.__str__()
    
    def forward_pass(self, input_data):
        x = self.layer1(input_data)
        x = torch.nn.functional.leaky_relu(x)
        x = self.layer2(x)
        x = torch.nn.functional.leaky_relu(x)
        x = self.layer3(x)
        x = torch.nn.functional.leaky_relu(x)
        x = self.layer4(x)
        x = torch.nn.functional.leaky_relu(x)
        x = self.layer5(x)
        return x

In [355]:
# initialize model
model = SingleAgentNN()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Check if GPU is available and if so, use it
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
print(f"Using device: {device}")

# Transfer the model to the GPU
model.to(device)


M = 1 #500
N = 10 #3000

M = 500
N = 2000

for episode in range(M):
    record = Counter()

    # create a loss function
    cur_state = torch.zeros(60).to(device)
    total_rewards = 0

    l_time = 0
    r_time = 0
    import time
    for timestep in range(N):
        print(timestep)
        # CPU part
        pre = time.time()
        actions = model.forward_pass(cur_state).to(device)
        next_state, reward = step(cur_state, actions)
        next_state = torch.tensor(next_state).to(device)
        post = time.time()

        l_time += post - pre

        # GPU part
        pre = time.time()
        total_rewards += reward
        # deep Q learning update
        expected_value = reward + 0.9 * torch.max(model.forward_pass(next_state))
        actual_value = torch.max(actions).float()

        loss = torch.nn.MSELoss()(expected_value, actual_value)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        cur_state = next_state
        post = time.time()
        r_time += post - pre

        # print(cur_state)

    print(total_rewards, l_time, r_time)
    print(record)

Using device: cuda:0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
2

KeyboardInterrupt: 

Above are implementation of a previous student's work.
Following will be the work by myself.

# Data Center
Used to 

In [165]:
STATE_SIZE = 28
QUERY_SIZE = 1
VALUE_SIZE = 4

JOB_SIZE = 2

In [7]:
# import necessary libraries
import torch
import torch.nn as nn

In [408]:
# Each data center holds two NN, one for state compression (Attention-model) and another for action selection (DQN), the following class represents the Attention model

# It has access to the full state of the data center and state representation of neighboring data centers

# State Compression Using Attention Model
class StateCompressor(nn.Module):
    def __init__(self, state_size, query_size, value_size, device="cpu"):
        super(StateCompressor, self).__init__()
        # might replace into a more appropriate encoder
        self.W_v_local = nn.Linear(state_size, value_size).to(device)
        # attention qkv
        self.W_q = nn.Linear(value_size, query_size).to(device)
        # value size add 1 to include extra encoding for the local state distingushment
        self.W_k = nn.Linear(value_size+1, query_size).to(device)
        self.W_v = nn.Linear(value_size+1, value_size).to(device)

        self.device = device

    def __str__(self):
        return ""
    
    def __repr__(self):
        return self.__str__()
    
    def forward_pass(self, local_state, remote_info):
        local_info = self.W_v_local(local_state.detach())
        full_info = torch.cat((local_info.detach().unsqueeze(0), remote_info.detach()), 0)
        full_info_with_encoding = torch.cat((full_info, torch.zeros(full_info.size(0), 1).to(self.device)), 1)
        full_info_with_encoding[0, -1] = 1


        # print("full_info", full_info.size())
        q = self.W_q(local_info)
        k = self.W_k(full_info_with_encoding)
        v = self.W_v(full_info_with_encoding)
        # print(q.size(), k.size(), v.size())
        x = torch.matmul(q, k.T)
        # print(x.size())
        x = torch.nn.functional.softmax(x)
        x = torch.matmul(x, v)

        return local_info + x

In [409]:
# brief test
state_compressor = StateCompressor(STATE_SIZE, QUERY_SIZE, VALUE_SIZE)
print(state_compressor.forward_pass(torch.zeros(STATE_SIZE), torch.zeros((2,4))))

tensor([0.4694, 0.1997, 0.2103, 0.2271], grad_fn=<AddBackward0>)


C:\Users\l\AppData\Local\Temp\ipykernel_29308\2647041914.py:43: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = torch.nn.functional.softmax(x)


In [410]:
# action selection using DQN

class DQN(nn.Module):
    def __init__(self, state_size, rep_size, device="cpu"):
        super(DQN, self).__init__()
        self.layer1 = nn.Linear(state_size+rep_size+JOB_SIZE+1, 512).to(device)
        self.layer2 = nn.Linear(512, 256).to(device)
        self.layer3 = nn.Linear(256, 256).to(device)
        self.layer4 = nn.Linear(256, 256).to(device)
        self.layer5 = nn.Linear(256, 1).to(device)
    
    def forward_pass(self, x):
        x = self.layer1(x)
        x = torch.nn.functional.leaky_relu(x)
        x = self.layer2(x)
        x = torch.nn.functional.leaky_relu(x)
        x = self.layer3(x)
        x = torch.nn.functional.leaky_relu(x)
        x = self.layer4(x)
        x = torch.nn.functional.leaky_relu(x)
        x = self.layer5(x)
        return x

In [411]:
# brief test
dqn = DQN(STATE_SIZE, VALUE_SIZE)
print(dqn.forward_pass(torch.zeros(STATE_SIZE + 4 + 2 + 1)))

tensor([0.0077], grad_fn=<ViewBackward0>)


With 2 Data Center, each with 5 machines and 6 queue length.
total feature nubmer is 2 * (5 * 2+6 * 3+2) = 60

which means that per machine, we have 5*2+6*3 = 28 state size

In [431]:
class DataCenter():
    def __init__(self, device):
        # self.data_center_id = data_center_id
        # self.machine_num = machine_num
        # self.queue_num = queue_num
        self.state = torch.zeros(STATE_SIZE).to(device)
        self.compressor = StateCompressor(STATE_SIZE, QUERY_SIZE, VALUE_SIZE, device=device)
        self.dqn = DQN(STATE_SIZE, VALUE_SIZE)
        self.representations = torch.zeros(VALUE_SIZE).to(device)

        self.device = device

        self.dqn_optimizer = torch.optim.Adam(self.dqn.parameters(), lr=0.001)
        self.compressor_optimizer = torch.optim.Adam(self.compressor.parameters(), lr=0.001)
    
    def update(self, delta):
        with torch.no_grad():
            # reward = gains from successful job allocation - losses from queueing delay
            reward = torch.tensor(0.0).to(self.device)
            # Separate machine states and queue states
            machines = self.state[:10].view(5, 2).clone()
            queues = self.state[10:].view(6, 3).clone()

            # Update machine states
            machines[:, 1] = torch.maximum(torch.zeros_like(machines[:, 1]), machines[:, 1] - delta)
            machines[machines[:, 1] == 0, 0] = 0

            # Find available machines and assign jobs from the queue
            for i in range(queues.size(0)):
                if queues[i, 0] > 0:
                    # Find first available machine
                    available_machine_index = torch.nonzero(machines[:, 0] == 0, as_tuple=False)
                    if available_machine_index.size(0) > 0:
                        first_available = available_machine_index[0].item()
                        machines[first_available, 0] = 1
                        machines[first_available, 1] = queues[i, 1]
                        reward += queues[i, 2]
                        queues[i, :] = 0
            
            queues[:, 2] = torch.maximum(torch.zeros_like(queues[:, 2]), queues[:, 2] - 0.1)

            # Merge the updated machine and queue states back into self.state
            self.state = torch.cat((machines.view(-1), queues.view(-1)))

            return reward
    
    def update_rep(self, remote_info):
        new_reps = self.compressor.forward_pass(self.state, remote_info)
        # print(new_reps.size(), self.representations.size())
        assert new_reps.size() == self.representations.size()
        self.representations = new_reps
    
    def get_q_values(self, reps, job):
        batch_input = torch.zeros((2, STATE_SIZE + VALUE_SIZE + JOB_SIZE + 1))
        batch_input[0] = torch.cat((self.state, self.representations, job, torch.ones(1).to(self.device)), 0)
        for i in range(reps.size(0)):
            batch_input[i+1] = torch.cat((self.state, reps[i], job, torch.zeros(1).to(self.device)), 0)
            # expand the concat into several instructions
        
        # batch_input[0, :STATE_SIZE] = self.state
        # # batch_input[0, STATE_SIZE:STATE_SIZE+VALUE_SIZE] = self.representations
        # batch_input[0, STATE_SIZE+VALUE_SIZE:STATE_SIZE+VALUE_SIZE+JOB_SIZE] = job

        batch_input.to(self.device)
        q_values = self.dqn.forward_pass(batch_input)
        return q_values

    # add job to the queue of the data center
    def add_job(self, job):
        state = self.state.clone()
        for i in range(6):
            if state[10+i*3] == 0:
                state[10+i*3] = 1
                state[10+i*3+1] = job[0]
                state[10+i*3+2] = job[1]
                break
        self.state = state


    
    # how to do this?
    def backprop(self):
        self.dqn_optimizer.step()
        self.compressor_optimizer.step()

        self.dqn_optimizer.zero_grad()
        self.compressor_optimizer.zero_grad()

In [432]:
# test
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

dc = DataCenter(device)
dc.state[0] = 1
dc.state[1] = 4
dc.state[2] = 1
dc.state[3] = 1
dc.state[10] = 1
dc.state[11] = 5
dc.state[12] = 12
dc.state[13] = 1
dc.state[14] = 4
dc.state[15] = 1

dc.update(0.1)
print(dc.state)

remote_info = torch.zeros((2, 4)).to(device)
dc.update_rep(remote_info)
print(dc.representations)

tensor([1.0000, 3.9000, 1.0000, 0.9000, 1.0000, 5.0000, 1.0000, 4.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000], device='cuda:0')
tensor([-1.1559, -2.4334, -0.1383,  0.5641], device='cuda:0',
       grad_fn=<AddBackward0>)


C:\Users\l\AppData\Local\Temp\ipykernel_29308\2647041914.py:43: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = torch.nn.functional.softmax(x)


In [433]:
class JobGenerator():
    def __init__(self, data_center_num) -> None:
        self.data_center_num = data_center_num

    def generate_job(self):
        jobs = []
        for i in range(self.data_center_num):
            seed = torch.rand(1).item()
            if seed < 0.4:
                jobs.append((8, 0.8))
            elif seed < 0.7:
                jobs.append((4, 0.4))
            else:
                jobs.append((2, 0.2))
        return [torch.tensor(j) for j in jobs]

In [434]:
# test
job_generator = JobGenerator(2)
print(job_generator.generate_job())

[tensor([8.0000, 0.8000]), tensor([2.0000, 0.2000])]


In [435]:
def epsilon_greedy(q_values, epsilon):
    action = None
    if torch.rand(1).item() < epsilon:
        action = torch.randint(0, q_values.size(0), (1, ))
    else:
        action = torch.argmax(q_values)
    q_value = q_values[action]
    return action, q_value

In [436]:
# Main Simulation

# configs
torch.autograd.set_detect_anomaly(True)


# Check if GPU is available and if so, use it
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
print(f"Using device: {device}")

# Create Data Centers
data_center_num = 2
dataCenter1 = DataCenter(device)
dataCenter2 = DataCenter(device)


M = 1 #500
N = 300 #3000

# M = 500
# N = 3000

for episode in range(M):
    # keep track of rewards
    total_rewards = 0

    # create a loss function
    dataCenter1.state = torch.zeros(STATE_SIZE).to(device)
    dataCenter2.state = torch.zeros(STATE_SIZE).to(device)
    total_rewards = 0

    jobGenerator = JobGenerator(data_center_num)

    l_time = 0
    r_time = 0
    import time

    jobs = jobGenerator.generate_job()

    # get initial actions
    q_values_1 = dataCenter1.get_q_values(dataCenter2.representations.unsqueeze(0), jobs[0].to(device))
    q_values_2 = dataCenter2.get_q_values(dataCenter1.representations.unsqueeze(0), jobs[1].to(device))

    action1, q_value_1 = epsilon_greedy(q_values_1, 0.1)
    action2, q_value_2 = epsilon_greedy(q_values_2, 0.1)

    for timestep in range(N):
        # update according to action
        pre = time.time()

        reward1 = 0
        reward2 = 0

        reward = torch.tensor(0.0).to(device)

        if action1 == 0:
            dataCenter1.add_job(jobs[0])
        else:
            jobs[0][1] -= 0.2
            dataCenter2.add_job(jobs[0])

        if action2 == 0:
            dataCenter2.add_job(jobs[1])
        else:
            jobs[1][1] -= 0.2
            dataCenter1.add_job(jobs[1])

        reward += dataCenter1.update(1)
        reward += dataCenter2.update(1)

        post = time.time()
        l_time += post - pre
        

        
        # update representations
        pre = time.time()
        jobs = jobGenerator.generate_job()

        # get the representations
        dataCenter1.update_rep(dataCenter2.representations.unsqueeze(0))
        dataCenter2.update_rep(dataCenter1.representations.unsqueeze(0))

        # get next actions
        new_q_values_1 = dataCenter1.get_q_values(dataCenter2.representations.unsqueeze(0), jobs[0].to(device))
        new_q_values_2 = dataCenter2.get_q_values(dataCenter1.representations.unsqueeze(0), jobs[1].to(device))

        new_action1, new_q_value_1 = epsilon_greedy(q_values_1, 0.1)
        new_action2, new_q_value_2 = epsilon_greedy(q_values_2, 0.1)

        post = time.time()
        r_time += post - pre

        # handle rewards
        total_rewards += reward
        reward1 = reward / 2
        reward2 = reward / 2

        print(f"{timestep}th loop", reward, total_rewards, dataCenter1.state, dataCenter1.representations)

        # backprop
        expected_value_1 = reward1 + 0.9 * torch.max(new_q_values_1)
        actual_value_1 = q_value_1.to(device)
        loss_1 = torch.nn.MSELoss()(expected_value_1, actual_value_1)
        loss_1.backward(retain_graph=True)

        expected_value_2 = reward2 + 0.9 * torch.max(new_q_values_2)
        actual_value_2 = q_value_2.to(device)
        loss_2 = torch.nn.MSELoss()(expected_value_2, actual_value_2)
        loss_2.backward(retain_graph=True)

        if episode % 10 == 0:
            dataCenter1.backprop()
            dataCenter2.backprop()

        # need to re-calculate the q_values to avoid issues in the backpropagation
        
        # get the representations
        dataCenter1.update_rep(dataCenter2.representations.unsqueeze(0))
        dataCenter2.update_rep(dataCenter1.representations.unsqueeze(0))

        # get initial actions
        q_values_1 = dataCenter1.get_q_values(dataCenter2.representations.unsqueeze(0), jobs[0].to(device))
        q_values_2 = dataCenter2.get_q_values(dataCenter1.representations.unsqueeze(0), jobs[1].to(device))

        action1, q_value_1 = epsilon_greedy(q_values_1, 0.1)
        action2, q_value_2 = epsilon_greedy(q_values_2, 0.1)



        # # GPU part
        # pre = time.time()
        # total_rewards += reward
        # # deep Q learning update
        # expected_value = reward + 0.9 * torch.max(model.forward_pass(next_state))
        # actual_value = torch.max(actions).float()

        # loss = torch.nn.MSELoss()(expected_value, actual_value)
        # loss.backward()
        # optimizer.step()
        # optimizer.zero_grad()
        # cur_state = next_state
        # post = time.time()
        # r_time += post - pre

        # print(cur_state)

    print(total_rewards, l_time, r_time)

Using device: cuda:0


C:\Users\l\AppData\Local\Temp\ipykernel_29308\2647041914.py:43: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = torch.nn.functional.softmax(x)
c:\Users\l\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


0th loop tensor(0.4000, device='cuda:0') tensor(0.4000, device='cuda:0') tensor([1., 4., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0') tensor([-0.5093, -0.5203, -0.3438, -0.7589], device='cuda:0',
       grad_fn=<AddBackward0>)
1th loop tensor(1.2000, device='cuda:0') tensor(1.6000, device='cuda:0') tensor([1., 3., 1., 4., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0') tensor([-0.2666, -0.7468, -1.1292,  0.6732], device='cuda:0',
       grad_fn=<AddBackward0>)
2th loop tensor(0.8000, device='cuda:0') tensor(2.4000, device='cuda:0') tensor([1., 2., 1., 3., 1., 8., 1., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0') tensor([ 1.2933, -1.3502,  0.3379, -0.0859], device='cuda:0',
       grad_fn=<AddBackward0>)
3th loop tensor(1., device='cuda:0') tensor(3.4000, device='cuda

c:\Users\l\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([1, 1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


18th loop tensor(1., device='cuda:0') tensor(9.5000, device='cuda:0') tensor([1., 8., 1., 3., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0') tensor([-1.2513, -1.6464, -1.0325, -1.0520], device='cuda:0',
       grad_fn=<AddBackward0>)
19th loop tensor(0.8000, device='cuda:0') tensor(10.3000, device='cuda:0') tensor([1., 7., 1., 2., 1., 2., 1., 8., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0') tensor([ 0.3410, -1.8725,  0.1544, -1.4830], device='cuda:0',
       grad_fn=<AddBackward0>)
20th loop tensor(0.4000, device='cuda:0') tensor(10.7000, device='cuda:0') tensor([1., 6., 1., 1., 1., 1., 1., 7., 1., 4., 0., 0., 0., 1., 2., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0') tensor([ 0.5834, -2.2409,  0.6006, -1.1236], device='cuda:0',
       grad_fn=<AddBackward0>)
21th loop tensor(0.6000, device='cuda:0') tensor(11.3000, devic

KeyboardInterrupt: 